Password match - get user ID

In [1]:
from services.user_dao import password_match, get_user_id

In [2]:
admin = "admin"
password = "admin"

In [3]:
if not password_match(admin, password):
    raise Exception("Login failed")

user_id = get_user_id(admin)
print(f"Login successful – User ID: {user_id}")

Login successful – User ID: 1


Retrieve all portfolios and transaction history related to this ID

In [19]:
from services.portfolio_dao import get_portfolios_by_user

In [20]:
portfolios = get_portfolios_by_user(user_id)
portfolio_id_to_name = {p.id: p.name for p in portfolios}

In [21]:
from services.transaction_dao import get_transactions_for_portfolio

In [22]:
portfolio_transactions = {
    p.id: get_transactions_for_portfolio(p.id)
    for p in portfolios
}

In [23]:
frontend_transactions = []
for pid, txs in portfolio_transactions.items():
    portfolio_name = portfolio_id_to_name.get(pid, "Unknown Portfolio")
    for tx in txs:
        frontend_transactions.append({
            "id": tx["id"],
            "portfolio": portfolio_name,
            "ticker": tx["ticker"],
            "type": tx["type"].title(),
            "quantity": tx["quantity"],
            "price": tx["price"],
            "dateTime": tx["executed_at"].strftime("%Y-%m-%d %H:%M:%S")
        })

In [24]:
frontend_transactions

[{'id': 1,
  'portfolio': 'Retirement Portfolio',
  'ticker': 'AAPL',
  'type': 'Buy',
  'quantity': 10,
  'price': 150.0,
  'dateTime': '2025-04-15 18:34:45'},
 {'id': 2,
  'portfolio': 'Retirement Portfolio',
  'ticker': 'GOOGL',
  'type': 'Buy',
  'quantity': 5,
  'price': 2800.0,
  'dateTime': '2025-04-15 18:34:45'},
 {'id': 5,
  'portfolio': 'Retirement Portfolio',
  'ticker': 'MSFT',
  'type': 'Buy',
  'quantity': 10,
  'price': 100.0,
  'dateTime': '2025-04-22 00:22:51'},
 {'id': 6,
  'portfolio': 'Retirement Portfolio',
  'ticker': 'AAPL',
  'type': 'Sell',
  'quantity': 5,
  'price': 150.0,
  'dateTime': '2025-04-22 00:28:24'},
 {'id': 8,
  'portfolio': 'Retirement Portfolio',
  'ticker': 'APPL',
  'type': 'Buy',
  'quantity': 10,
  'price': 100.0,
  'dateTime': '2025-04-22 00:31:22'},
 {'id': 9,
  'portfolio': 'Retirement Portfolio',
  'ticker': 'AAPL',
  'type': 'Buy',
  'quantity': 10,
  'price': 150.0,
  'dateTime': '2025-04-30 02:32:21'},
 {'id': 10,
  'portfolio': 'Retir

Fetch latest market data related to our portfolio stocks

In [10]:
from services.market_service import fetch_market_data

In [11]:
stock_lookup = {s['ticker']: s for s in fetch_market_data()}

Retreive all Stocks (positions) related to our portfolios

In [12]:
from services.position_dao import get_positions_by_portfolio

In [13]:
portfolio_summary = []

for portfolio in portfolios:
    positions = get_positions_by_portfolio(portfolio.id)

    positions_list = []
    for p in positions:
        stock_info = stock_lookup.get(p.ticker, {})
        positions_list.append({
            "id": p.id,
            "ticker": p.ticker,
            "quantity": p.quantity,
            "average_price": float(p.average_price),
            "created_at": p.created_at.isoformat(),
            "updated_at": p.updated_at.isoformat(),
            "current_price": stock_info.get("price"),
            "percent_change": stock_info.get("changePercent")
        })

    portfolio_summary.append({
        "portfolio_id": portfolio.id,
        "portfolio_name": portfolio.name,
        "strategy": portfolio.strategy.value,
        "positions": positions_list
    })

In [14]:
import json

In [15]:
json_result = json.dumps(portfolio_summary, indent=2)
print(json_result)

[
  {
    "portfolio_id": 1,
    "portfolio_name": "Retirement Portfolio",
    "strategy": "long_term",
    "positions": [
      {
        "id": 1,
        "ticker": "AAPL",
        "quantity": 84,
        "average_price": 160.72,
        "created_at": "2025-04-15T18:34:24",
        "updated_at": "2025-04-30T03:11:42",
        "current_price": 211.21,
        "percent_change": 0.51
      },
      {
        "id": 2,
        "ticker": "GOOGL",
        "quantity": 5,
        "average_price": 2800.0,
        "created_at": "2025-04-15T18:34:24",
        "updated_at": "2025-04-15T18:34:24",
        "current_price": 160.16,
        "percent_change": -0.28
      },
      {
        "id": 7,
        "ticker": "META",
        "quantity": 11,
        "average_price": 175.0,
        "created_at": "2025-04-30T03:15:06",
        "updated_at": "2025-04-30T03:15:06",
        "current_price": 554.44,
        "percent_change": 0.85
      },
      {
        "id": 4,
        "ticker": "MSFT",
        "quan

Buy a stock and save the transaction

In [16]:
from services.transaction_dao import add_transaction
from services.position_dao import buy_stock
from models.transactions import TransactionType

In [17]:
portfolio_id = 1
ticker = "META"
quantity = 11
price = 175.00

In [18]:
message = buy_stock(portfolio_id, ticker, quantity, price)
print(message)
add_transaction(portfolio_id, ticker, quantity, price, TransactionType.buy)

Successfully bought 11 of META stock at $175.00


Sell a stock and save teh transaction

In [ ]:
from services.position_dao import sell_stock
from services.transaction_dao import add_transaction
from models.transactions import TransactionType

In [2]:
portfolio_id = 1
ticker = "META"
quantity = 11
price = 175.00

In [ ]:
sell_stock(portfolio_id, ticker, quantity, price)
add_transaction(portfolio_id, ticker, quantity, price, TransactionType.sell)

'Successfully sold 11 of META stock at $175.00'